In [1]:
import pandas as pd

# Load merged mBERT output
df = pd.read_csv("mBERT_Merged_Multilingual.csv")

# Check if all three predictions agree
df["All_Agree"] = (
    (df["mBERT_EN"] == df["mBERT_FA"]) &
    (df["mBERT_EN"] == df["mBERT_IT"])
)

# Calculate agreement rate
agreement_rate = df["All_Agree"].mean() * 100
print(f"✅ Full agreement across EN, FA, IT: {agreement_rate:.2f}%")

✅ Full agreement across EN, FA, IT: 3.33%


In [2]:
df["Disagree"] = ~df["All_Agree"]
disagreements_by_domain = df[df["Disagree"]].groupby("Domain_EN").size()
print(disagreements_by_domain)

Domain_EN
Activism & Social Change    5
Arts & Culture              4
Business & Economics        5
Caregiving & Parenting      5
Education                   5
Healthcare                  5
Hospitality & Service       5
Journalism & Media          5
Law & Justice               5
Politics & Leadership       5
Religion & Spirituality     4
Science & Technology        5
dtype: int64


In [3]:
import pandas as pd

# Load the merged mBERT output
df = pd.read_csv("mBERT_Merged_Multilingual.csv")

# Add Disagreement_Type column for mBERT
df["Disagreement_Type"] = df.apply(lambda row: 
    "FA diverged" if row["mBERT_EN"] == row["mBERT_IT"] != row["mBERT_FA"] else
    "IT diverged" if row["mBERT_EN"] == row["mBERT_FA"] != row["mBERT_IT"] else
    "EN diverged" if row["mBERT_FA"] == row["mBERT_IT"] != row["mBERT_EN"] else
    "All different", axis=1
)

# Print the disagreement breakdown
print(df["Disagreement_Type"].value_counts())


Disagreement_Type
All different    23
FA diverged      14
EN diverged      13
IT diverged      10
Name: count, dtype: int64


In [4]:
print("Option frequency in FA:")
print(df["mBERT_FA"].value_counts(normalize=True).round(2) * 100)

Option frequency in FA:
mBERT_FA
B    28.0
D    28.0
C    25.0
A    18.0
Name: proportion, dtype: float64


In [5]:
import pandas as pd

# Load your aligned mBERT results
df = pd.read_csv("mBERT_Merged_Multilingual.csv")

# Identify disagreement rows (not all 3 agree)
df["Disagreement"] = (df["mBERT_EN"] != df["mBERT_FA"]) | \
                     (df["mBERT_EN"] != df["mBERT_IT"]) | \
                     (df["mBERT_FA"] != df["mBERT_IT"])

# Export only the rows with disagreement
df_disagreement = df[df["Disagreement"] == True]
df_disagreement.to_csv("mBERT_Disagreements.csv", index=False)
print(f"✅ Exported {len(df_disagreement)} disagreement cases to mBERT_Disagreements.csv")

# Count disagreement cases per domain
disagreement_by_domain = df_disagreement["Domain_EN"].value_counts().reset_index()
disagreement_by_domain.columns = ["Domain", "Disagreement_Count"]
disagreement_by_domain.to_csv("mBERT_Disagreement_By_Domain.csv", index=False)
print("✅ Saved domain-wise disagreement counts to mBERT_Disagreement_By_Domain.csv")

# (Optional) Show most frequent disagreement domains in console
print("\n📊 Top domains with most disagreement:")
print(disagreement_by_domain.head(10))


✅ Exported 58 disagreement cases to mBERT_Disagreements.csv
✅ Saved domain-wise disagreement counts to mBERT_Disagreement_By_Domain.csv

📊 Top domains with most disagreement:
                     Domain  Disagreement_Count
0     Politics & Leadership                   5
1                 Education                   5
2                Healthcare                   5
3      Science & Technology                   5
4      Business & Economics                   5
5             Law & Justice                   5
6        Journalism & Media                   5
7  Activism & Social Change                   5
8    Caregiving & Parenting                   5
9     Hospitality & Service                   5


In [6]:
import pandas as pd
from collections import Counter

# Load merged results file for mBERT
df = pd.read_csv("mBERT_Merged_Multilingual.csv")

# Check full agreement
df["Full_Agreement"] = (df["mBERT_EN"] == df["mBERT_FA"]) & (df["mBERT_EN"] == df["mBERT_IT"])

# Classify type of disagreement
def classify_disagreement(row):
    choices = {row["mBERT_EN"], row["mBERT_FA"], row["mBERT_IT"]}
    if len(choices) == 1:
        return "All same"
    elif len(choices) == 3:
        return "All different"
    elif row["mBERT_EN"] != row["mBERT_FA"] and row["mBERT_EN"] == row["mBERT_IT"]:
        return "FA diverged"
    elif row["mBERT_EN"] != row["mBERT_IT"] and row["mBERT_EN"] == row["mBERT_FA"]:
        return "IT diverged"
    elif row["mBERT_EN"] != row["mBERT_FA"] and row["mBERT_FA"] == row["mBERT_IT"]:
        return "EN diverged"
    else:
        return "Other"

df["Disagreement_Type"] = df.apply(classify_disagreement, axis=1)

# Count full agreement
agreement_count = df["Full_Agreement"].sum()
total = len(df)
agreement_pct = agreement_count / total * 100

# Disagreement breakdown
disagreement_summary = df["Disagreement_Type"].value_counts().to_frame("count")

# Frequency of choices in Persian
fa_freq = df["mBERT_FA"].value_counts(normalize=True).rename("proportion").to_frame()

# Domain-wise disagreement
domain_disagreements = df[df["Disagreement_Type"] != "All same"]["Domain_EN"].value_counts().to_frame("count")

# Display everything
print("🔎 Full Agreement Rate:")
print(f"{agreement_count}/{total} prompts ({agreement_pct:.2f}%)\n")

print("🧩 Disagreement Types:")
print(disagreement_summary, "\n")

print("🗳️ Option Frequency in Persian (mBERT_FA):")
print(fa_freq, "\n")

print("🌍 Domain-wise Disagreements:")
print(domain_disagreements)


🔎 Full Agreement Rate:
2/60 prompts (3.33%)

🧩 Disagreement Types:
                   count
Disagreement_Type       
All different         21
FA diverged           14
EN diverged           13
IT diverged           10
All same               2 

🗳️ Option Frequency in Persian (mBERT_FA):
          proportion
mBERT_FA            
B           0.283333
D           0.283333
C           0.250000
A           0.183333 

🌍 Domain-wise Disagreements:
                          count
Domain_EN                      
Politics & Leadership         5
Education                     5
Healthcare                    5
Science & Technology          5
Business & Economics          5
Law & Justice                 5
Journalism & Media            5
Activism & Social Change      5
Caregiving & Parenting        5
Hospitality & Service         5
Arts & Culture                4
Religion & Spirituality       4


In [1]:
import pandas as pd
import numpy as np

def bootstrap_mbert_agreement_CI(filepath, prefix="mBERT", num_bootstrap=1000, confidence=0.95):
    df = pd.read_csv(filepath)
    en, fa, it = f"{prefix}_EN", f"{prefix}_FA", f"{prefix}_IT"
    df["Full_Agreement"] = (df[en] == df[fa]) & (df[en] == df[it])

    bootstrapped_agreements = []
    for _ in range(num_bootstrap):
        sample = df.sample(frac=1.0, replace=True)
        agreement_pct = sample["Full_Agreement"].mean() * 100
        bootstrapped_agreements.append(agreement_pct)

    lower_bound = np.percentile(bootstrapped_agreements, (1 - confidence) / 2 * 100)
    upper_bound = np.percentile(bootstrapped_agreements, (1 + confidence) / 2 * 100)
    mean_agreement = np.mean(bootstrapped_agreements)

    print(f"Bootstrapped {int(confidence*100)}% CI for mBERT agreement:")
    print(f"Mean agreement: {mean_agreement:.2f}%")
    print(f"{int(confidence*100)}% Confidence Interval: [{lower_bound:.2f}%, {upper_bound:.2f}%]")

# Example usage:
bootstrap_mbert_agreement_CI("mBERT_Merged_Multilingual.csv")


Bootstrapped 95% CI for mBERT agreement:
Mean agreement: 3.23%
95% Confidence Interval: [0.00%, 8.33%]


In [1]:
import pandas as pd
import numpy as np

def bootstrap_mbert_disagreement_CI(filepath, prefix="mBERT", num_bootstrap=1000, confidence=0.95):
    df = pd.read_csv(filepath)
    en, fa, it = f"{prefix}_EN", f"{prefix}_FA", f"{prefix}_IT"
    # Disagreement means not all three agree
    df["Disagreement"] = ~((df[en] == df[fa]) & (df[en] == df[it]))

    bootstrapped_disagreements = []
    for _ in range(num_bootstrap):
        sample = df.sample(frac=1.0, replace=True)
        disagreement_pct = sample["Disagreement"].mean() * 100
        bootstrapped_disagreements.append(disagreement_pct)

    lower_bound = np.percentile(bootstrapped_disagreements, (1 - confidence) / 2 * 100)
    upper_bound = np.percentile(bootstrapped_disagreements, (1 + confidence) / 2 * 100)
    mean_disagreement = np.mean(bootstrapped_disagreements)

    print(f"Bootstrapped {int(confidence*100)}% CI for {prefix} disagreement:")
    print(f"Mean disagreement: {mean_disagreement:.2f}%")
    print(f"{int(confidence*100)}% Confidence Interval: [{lower_bound:.2f}%, {upper_bound:.2f}%]")

# Example usage:
bootstrap_mbert_disagreement_CI("mBERT_Merged_Multilingual.csv")


Bootstrapped 95% CI for mBERT disagreement:
Mean disagreement: 96.63%
95% Confidence Interval: [91.67%, 100.00%]
